In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
df = pd.read_csv('combined_data.csv')
df.head()

,sentiment,Text
0,sadness,@tiffanylue i know i was listenin to bad habi...
1,sadness,Layin n bed with a headache ughhhh...waitin o...
2,sadness,Funeral ceremony...gloomy friday...
3,happy,wants to hang out with friends SOON!
4,neutral,@dannycastillo We want to trade with someone w...


In [3]:
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import re

stopWords = stopwords.words("english")

def cleanText(text):
    
    output = ""
    
    
    for word in text.split():
        cleanWord = re.sub("[^a-zA-Z]",  # Search for all non-letters
                          " ",          # Replace all non-letters with spaces
                          str(word))
        if cleanWord in stopWords:
            pass
        else:
            output = output + " " + cleanWord
    return output

In [4]:
df.Text = df.Text.apply(lambda x: cleanText(x))

In [5]:
from sklearn.model_selection import train_test_split

#overall train and test
X_train, X_test = train_test_split(df, test_size=0.2, random_state=42)

In [6]:
#fear df
fearDf = df[df['sentiment'] == 'fear']

#happy df 
happyDf = df[df['sentiment'] == 'happy']


#sadness df
sadDf = df[df['sentiment'] == 'sadness']


#anger df
angerDf = df[df['sentiment'] == 'anger']


In [7]:
from sklearn import svm
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import classification_report
from sklearn.tree import DecisionTreeClassifier

clf = svm.SVC(kernel='linear')

In [8]:
import time

#Fear vs Happy df
fearHappyDf = pd.concat([fearDf, happyDf], axis=0)
fh_train, fh_test = train_test_split(fearHappyDf, test_size=0.2, random_state=42)

vectorizer = TfidfVectorizer(min_df = 5,
                             max_df = 0.8,
                             sublinear_tf = True,
                             use_idf = True)
train_vectors = vectorizer.fit_transform(fh_train['Text'])
test_vectors = vectorizer.transform(fh_test['Text'])

# Perform classification with SVM, kernel=linear
classifier_linear = svm.SVC(kernel='linear')
t0 = time.time()
classifier_linear.fit(train_vectors, fh_train['sentiment'])
t1 = time.time()
prediction_linear = classifier_linear.predict(test_vectors)
t2 = time.time()
time_linear_train = t1-t0
time_linear_predict = t2-t1

# results
print("Training time: %fs; Prediction time: %fs" % (time_linear_train, time_linear_predict))
report = classification_report(fh_test['sentiment'], prediction_linear, output_dict=True)
print('fear: ', report['fear'])
print('happy: ', report['happy'])






Training time: 41.703556s; Prediction time: 5.628467s
fear:  {'precision': 0.7274909963985594, 'recall': 0.7406049495875344, 'f1-score': 0.733989401968206, 'support': 3273}
happy:  {'precision': 0.6757066462948816, 'recall': 0.6608143444153903, 'f1-score': 0.6681775259678943, 'support': 2677}


In [9]:
print(train_vectors)

  (0, 757)	0.5574552518645396
  (0, 1474)	0.13794736567791144
  (0, 1493)	0.45184057647492926
  (0, 1593)	0.4609231373578298
  (0, 2391)	0.20648217510612077
  (0, 3228)	0.45931387169737353
  (1, 132)	0.2580909629977318
  (1, 647)	0.268300325401973
  (1, 1727)	0.22937034368949272
  (1, 1740)	0.37616425650418966
  (1, 2062)	0.14836162054488716
  (1, 2313)	0.20596169347501156
  (1, 2391)	0.13729428289678108
  (1, 2962)	0.35066355668103155
  (1, 3077)	0.3033471952604451
  (1, 3906)	0.27429436449159145
  (1, 4097)	0.20760003328694396
  (1, 4468)	0.3921288043090607
  (1, 4573)	0.1871113601110105
  (1, 4625)	0.24883186031127622
  (2, 1566)	0.6015599968409576
  (2, 2287)	0.3201704145069095
  (2, 2339)	0.6217442782349224
  (2, 3391)	0.38607062612599324
  (3, 1476)	0.21872513481831346
  :	:
  (23796, 2085)	0.2728466283604734
  (23796, 2336)	0.3250677143436125
  (23796, 3260)	0.2957194020467564
  (23796, 3403)	0.33401181379810435
  (23796, 4144)	0.19715051434419506
  (23796, 4227)	0.3539807368494

In [10]:
import time

#Sadness vs Happy df
sadHappyDf = pd.concat([sadDf, happyDf], axis=0)
sh_train, sh_test = train_test_split(sadHappyDf, test_size=0.2, random_state=42)

vectorizer = TfidfVectorizer(min_df = 5,
                             max_df = 0.8,
                             sublinear_tf = True,
                             use_idf = True)
train_vectors = vectorizer.fit_transform(sh_train['Text'])
test_vectors = vectorizer.transform(sh_test['Text'])

# Perform classification with SVM, kernel=linear
classifier_linear = svm.SVC(kernel='linear')
t0 = time.time()
classifier_linear.fit(train_vectors, sh_train['sentiment'])
t1 = time.time()
prediction_linear = classifier_linear.predict(test_vectors)
t2 = time.time()
time_linear_train = t1-t0
time_linear_predict = t2-t1

# results
print("Training time: %fs; Prediction time: %fs" % (time_linear_train, time_linear_predict))
report = classification_report(sh_test['sentiment'], prediction_linear, output_dict=True)
print('sadness: ', report['sadness'])
print('happy: ', report['happy'])

Training time: 17.637299s; Prediction time: 2.869711s
sadness:  {'precision': 0.7838765008576329, 'recall': 0.6984207845134998, 'f1-score': 0.7386853448275863, 'support': 1963}
happy:  {'precision': 0.7967032967032966, 'recall': 0.859896219421794, 'f1-score': 0.8270944741532976, 'support': 2698}


In [11]:
#Sadness vs Anger df
sadAngryDf = pd.concat([sadDf, angerDf], axis=0)
sa_train, sa_test = train_test_split(sadAngryDf, test_size=0.2, random_state=42)

vectorizer = TfidfVectorizer(min_df = 5,
                             max_df = 0.8,
                             sublinear_tf = True,
                             use_idf = True)
train_vectors = vectorizer.fit_transform(sa_train['Text'])
test_vectors = vectorizer.transform(sa_test['Text'])

# Perform classification with SVM, kernel=linear
classifier_linear = svm.SVC(kernel='linear')
t0 = time.time()
classifier_linear.fit(train_vectors, sa_train['sentiment'])
t1 = time.time()
prediction_linear = classifier_linear.predict(test_vectors)
t2 = time.time()
time_linear_train = t1-t0
time_linear_predict = t2-t1

# results
print("Training time: %fs; Prediction time: %fs" % (time_linear_train, time_linear_predict))
report = classification_report(sa_test['sentiment'], prediction_linear, output_dict=True)
print('sadness: ', report['sadness'])
print('anger: ', report['anger'])

Training time: 5.766484s; Prediction time: 0.920254s
sadness:  {'precision': 0.8230668414154653, 'recall': 0.9467336683417086, 'f1-score': 0.8805795746669783, 'support': 1990}
anger:  {'precision': 0.78099173553719, 'recall': 0.4827586206896552, 'f1-score': 0.5966850828729282, 'support': 783}


In [12]:
from sklearn.feature_extraction.text import CountVectorizer
import gensim

bow_vectorizer = CountVectorizer(max_df = 0.9, min_df = 2, max_features = 1000, stop_words = 'english')
bow = bow_vectorizer.fit_transform(fh_train['Text'])
bow.shape

(23799, 1000)

In [13]:
happyVsRest = pd.read_csv('combined_data.csv')

happyVsRest['sentiment'] = np.where((happyVsRest.sentiment != 'happy'),'not happy', happyVsRest.sentiment)

happyVsRest.head()

,sentiment,Text
0,not happy,@tiffanylue i know i was listenin to bad habi...
1,not happy,Layin n bed with a headache ughhhh...waitin o...
2,not happy,Funeral ceremony...gloomy friday...
3,happy,wants to hang out with friends SOON!
4,not happy,@dannycastillo We want to trade with someone w...


In [14]:
#Non Happy vs Happy df
hnh_train, hnh_test = train_test_split(happyVsRest, test_size=0.2, random_state=42)

vectorizer = TfidfVectorizer(min_df = 5,
                             max_df = 0.8,
                             sublinear_tf = True,
                             use_idf = True)
train_vectors = vectorizer.fit_transform(hnh_train['Text'])
test_vectors = vectorizer.transform(hnh_test['Text'])

# Perform classification with SVM, kernel=linear
classifier_linear = svm.SVC(kernel='linear')
t0 = time.time()
classifier_linear.fit(train_vectors, hnh_train['sentiment'])
t1 = time.time()
prediction_linear = classifier_linear.predict(test_vectors)
t2 = time.time()
time_linear_train = t1-t0
time_linear_predict = t2-t1

# results
print("Training time: %fs; Prediction time: %fs" % (time_linear_train, time_linear_predict))
report = classification_report(hnh_test['sentiment'], prediction_linear, output_dict=True)
print('not happy: ', report['not happy'])
print('happy: ', report['happy'])

Training time: 371.668643s; Prediction time: 31.592927s
not happy:  {'precision': 0.7950072420550396, 'recall': 0.9780922431865828, 'f1-score': 0.8770973351506322, 'support': 9540}
happy:  {'precision': 0.6234234234234234, 'recall': 0.1257267441860465, 'f1-score': 0.20925309948593893, 'support': 2752}


In [15]:
fearVsRest = pd.read_csv('combined_data.csv')

fearVsRest['sentiment'] = np.where((fearVsRest.sentiment != 'fear'),'not fear', fearVsRest.sentiment)

fearVsRest.head()

,sentiment,Text
0,not fear,@tiffanylue i know i was listenin to bad habi...
1,not fear,Layin n bed with a headache ughhhh...waitin o...
2,not fear,Funeral ceremony...gloomy friday...
3,not fear,wants to hang out with friends SOON!
4,not fear,@dannycastillo We want to trade with someone w...


In [16]:
#Non Fear vs Fear df
fnf_train, fnf_test = train_test_split(fearVsRest, test_size=0.2, random_state=42)

vectorizer = TfidfVectorizer(min_df = 5,
                             max_df = 0.8,
                             sublinear_tf = True,
                             use_idf = True)
train_vectors = vectorizer.fit_transform(fnf_train['Text'])
test_vectors = vectorizer.transform(fnf_test['Text'])

# Perform classification with SVM, kernel=linear
classifier_linear = svm.SVC(kernel='linear')
t0 = time.time()
classifier_linear.fit(train_vectors, fnf_train['sentiment'])
t1 = time.time()
prediction_linear = classifier_linear.predict(test_vectors)
t2 = time.time()
time_linear_train = t1-t0
time_linear_predict = t2-t1

# results
print("Training time: %fs; Prediction time: %fs" % (time_linear_train, time_linear_predict))
report = classification_report(fnf_test['sentiment'], prediction_linear, output_dict=True)
print('not fear: ', report['not fear'])
print('fear: ', report['fear'])

Training time: 502.315996s; Prediction time: 34.620466s
not fear:  {'precision': 0.7567336468576314, 'recall': 0.9809354910219463, 'f1-score': 0.8543708065839647, 'support': 9022}
fear:  {'precision': 0.711892797319933, 'recall': 0.12996941896024464, 'f1-score': 0.21980863718644944, 'support': 3270}


In [19]:
sadVsRest = pd.read_csv('combined_data.csv')

sadVsRest['sadness'] = np.where((sadVsRest.sentiment != 'sadness'),'not sadness', sadVsRest.sentiment)

sadVsRest.head()

#Non Sadness vs Sadness df
sns_train, sns_test = train_test_split(sadVsRest, test_size=0.2, random_state=42)

vectorizer = TfidfVectorizer(min_df = 5,
                             max_df = 0.8,
                             sublinear_tf = True,
                             use_idf = True)
train_vectors = vectorizer.fit_transform(sns_train['Text'])
test_vectors = vectorizer.transform(sns_test['Text'])

# Perform classification with SVM, kernel=linear
classifier_linear = svm.SVC(kernel='linear')
t0 = time.time()
classifier_linear.fit(train_vectors, sns_train['sentiment'])
t1 = time.time()
prediction_linear = classifier_linear.predict(test_vectors)
t2 = time.time()
time_linear_train = t1-t0
time_linear_predict = t2-t1

# results
print("Training time: %fs; Prediction time: %fs" % (time_linear_train, time_linear_predict))
report = classification_report(sns_test['sentiment'], prediction_linear, output_dict=True)

print('sadness: ', report['sadness'])

Training time: 406.767569s; Prediction time: 61.910850s
sadness:  {'precision': 0.4468814256339959, 'recall': 0.33487416538264, 'f1-score': 0.3828537874339401, 'support': 1947}


In [21]:
angerVsRest = pd.read_csv('combined_data.csv')

angerVsRest['anger'] = np.where((angerVsRest.sentiment != 'anger'),'not anger', angerVsRest.sentiment)

angerVsRest.head()

ana_train, ana_test = train_test_split(angerVsRest, test_size=0.2, random_state=42)

vectorizer = TfidfVectorizer(min_df = 5,
                             max_df = 0.8,
                             sublinear_tf = True,
                             use_idf = True)
train_vectors = vectorizer.fit_transform(ana_train['Text'])
test_vectors = vectorizer.transform(ana_test['Text'])

# Perform classification with SVM, kernel=linear
classifier_linear = svm.SVC(kernel='linear')
t0 = time.time()
classifier_linear.fit(train_vectors, ana_train['sentiment'])
t1 = time.time()
prediction_linear = classifier_linear.predict(test_vectors)
t2 = time.time()
time_linear_train = t1-t0
time_linear_predict = t2-t1

# results
print("Training time: %fs; Prediction time: %fs" % (time_linear_train, time_linear_predict))
report = classification_report(ana_test['sentiment'], prediction_linear, output_dict=True)

print('anger: ', report['anger'])

Training time: 439.986216s; Prediction time: 69.803237s
anger:  {'precision': 0.48905109489051096, 'recall': 0.2554002541296061, 'f1-score': 0.33555926544240405, 'support': 787}
